# Summarize tasking requests

NASA CSDAP Assigned per-user quotas that are not tracked by Capella. This notebook summarizes individual orders and deducts from an assigned quota to keep track

In [1]:
# Keep track of module versions
#%pip install watermark
%load_ext watermark

In [2]:
import capella_console_client
import geopandas as gpd
import pandas as pd
import json

In [3]:
%watermark -iv

capella_console_client: 0.10.3
json                  : 2.0.9
pandas                : 2.0.3
IPython               : 8.14.0
geopandas             : 0.13.2



In [4]:
with open('/home/jovyan/capella/credentials.json') as f: # USER: Input path to credentials.json file.
    data = json.load(f)
    username = data['username']
    password = data['password']

client =  capella_console_client.CapellaConsoleClient(username, password)

In [5]:
# QUOTA
mindex = pd.MultiIndex.from_tuples([('spotlight', '1_day'),
                                    ('site', '1_day'),
                                    ('stripmap', '1_day')],
                                   names=['collect_mode', 'collectionTier'])

quota = pd.Series([30,30,30],index=mindex)
quota

collect_mode  collectionTier
spotlight     1_day             30
site          1_day             30
stripmap      1_day             30
dtype: int64

In [6]:
gf = gpd.GeoDataFrame.from_features(client.list_tasking_requests())

In [7]:
total_requests = len(gf)
print('Total Requests:', total_requests)

Total Requests: 39


In [8]:
# Remove cancelled orders
gf['final_status'] = gf['statusHistory'].apply(lambda x: x[0]['code'])

In [9]:
print('Cancelled:', (gf.final_status == 'canceled').sum())

Cancelled: 11


In [10]:
gf.final_status.value_counts()

final_status
completed    14
rejected     12
canceled     11
anomaly       1
review        1
Name: count, dtype: int64

In [11]:
gf['collect_mode'] = gf['collectConstraints'].str['collectMode']

In [12]:
# Only completed requests are counted
completed = gf[gf.final_status == 'completed']
result = completed.groupby(['collect_mode', 'collectionTier'])['geometry'].count()
result

collect_mode  collectionTier
spotlight     1_day             8
              background        1
stripmap      1_day             5
Name: geometry, dtype: int64

In [13]:
# REMAINING
quota - result

collect_mode  collectionTier
site          1_day              NaN
spotlight     1_day             22.0
              background         NaN
stripmap      1_day             25.0
dtype: float64